In [1]:
K, M = map(int, input("Enter K and M :").split())
def K_mers(read, K):
    bins = {}
    for idx in range(len(read)-K+1):         # total number of k-mers = L - K + 1 ; L = len(read)
        k_mer = read[idx:idx+K]              # slicing the read to get k-mer
        if k_mer not in bins:
            bins[k_mer]=[]
        bins[k_mer].append(idx)
    print(bins)
    return bins

def M_mers(read, M):
    m_mers = {'A':[], 'C':[], 'G':[], 'T':[]}

    for i in range(len(read)-M+1):
        m_mers[read[i]].append(read[i:i+M])
    
    print("read :", read)
    print(m_mers)
    return m_mers

def get_K_mer(read):
    print("Read :",read)
    
    K_mers(read, K)

def get_M_mer(read):
    M_mers(read, M)
    
read = [">SRR094926.sra.1", "TGGCGTCATTTTCCCCAATCA", ">SRR094926.sra.2", "TAGACTGTTGAAAGTGAACAAAACTGATGA"]

if __name__=="__main__":
    
    k_mer = list(map(get_K_mer, filter(lambda read: read[0]!=">", read)))
    print('M-mers -')
    m_mer = list(map(get_M_mer, filter(lambda read: read[0]!=">", read)))

Enter K and M :5 3
Read : TGGCGTCATTTTCCCCAATCA
{'TGGCG': [0], 'GGCGT': [1], 'GCGTC': [2], 'CGTCA': [3], 'GTCAT': [4], 'TCATT': [5], 'CATTT': [6], 'ATTTT': [7], 'TTTTC': [8], 'TTTCC': [9], 'TTCCC': [10], 'TCCCC': [11], 'CCCCA': [12], 'CCCAA': [13], 'CCAAT': [14], 'CAATC': [15], 'AATCA': [16]}
Read : TAGACTGTTGAAAGTGAACAAAACTGATGA
{'TAGAC': [0], 'AGACT': [1], 'GACTG': [2], 'ACTGT': [3], 'CTGTT': [4], 'TGTTG': [5], 'GTTGA': [6], 'TTGAA': [7], 'TGAAA': [8], 'GAAAG': [9], 'AAAGT': [10], 'AAGTG': [11], 'AGTGA': [12], 'GTGAA': [13], 'TGAAC': [14], 'GAACA': [15], 'AACAA': [16], 'ACAAA': [17], 'CAAAA': [18], 'AAAAC': [19], 'AAACT': [20], 'AACTG': [21], 'ACTGA': [22], 'CTGAT': [23], 'TGATG': [24], 'GATGA': [25]}
M-mers -
read : TGGCGTCATTTTCCCCAATCA
{'A': ['ATT', 'AAT', 'ATC'], 'C': ['CGT', 'CAT', 'CCC', 'CCC', 'CCA', 'CAA'], 'G': ['GGC', 'GCG', 'GTC'], 'T': ['TGG', 'TCA', 'TTT', 'TTT', 'TTC', 'TCC', 'TCA']}
read : TAGACTGTTGAAAGTGAACAAAACTGATGA
{'A': ['AGA', 'ACT', 'AAA', 'AAG', 'AGT', 'AAC', 

In [1]:
def charToTwobit(char: str):
    switcher = {
        'A' : 0x0,
        'C' : 0x1,
        'G' : 0x2,
        'T' : 0x3,
        'U' : 0x3
    }
    return switcher.get(char, "Invalid Nucleotide")

def twobitToChar(byte: bytes):
    switcher = {
        0 : 'A',
        1 : 'C',
        2 : 'G',
        3 : 'T'
    }
    return switcher.get(byte, "Invalid Value")

# convert into bits and gives direct priority
def encode(motif):
    code = {'A':'00','C':'01','G':'10','T':'11'}
    byteCode = ''.join(map(lambda x : code[x], motif))
    return int(byteCode, 2)

In [2]:
# for example
encode("AAC"), encode("AAA")

(1, 0)

#### Features => [patter, rank, valid]
####       Motif => [position, Feature]

In [3]:
class Features:
    def __init__(self, pat: int, rank: int, valid: bool):
        self.pattern = pat
        self.rank = rank
        self.valid = valid

    def __repr__(self):
        return '[%s, %s, %s]'%(self.pattern, self.rank, self.valid)
    
class Motif:
    def __init__(self, pos, feature):
        self.position = pos
        self.feature = feature

    def __repr__(self):
        return '[%s, (%s, %s)]' %(self.position, self.feature.pattern, self.feature.rank)

In [4]:
class MotifSpace:
    all1mersDNA = ("A", "C", "G", "T")
    all1mersRNA = ("A", "C", "G", "U")

    def __init__(self, length, rna=False):
        self.width = length
        self.rna = rna

        maxMotifs = 4 << (length * 2 - 2)
        self.priorityLookUp = [i for i in range(maxMotifs)]

    def motifsOfLength(self):
        bases = self.all1mersRNA if self.rna else self.all1mersDNA
        
        def generate(prefix, length):
            if (length == 0):
                yield prefix
                return

            for base in bases:
                yield from generate(prefix + base, length-1)

        return generate("", self.width)

    def scanner(self):
        return ShiftScanner(self)
    
    @staticmethod
    def priorityOf(mk):
        return encode(mk)

    def create(self, pattern, pos):
        return Motif(pos, Features(pattern, self.priorityOf(pattern), True))

In [31]:
# required parameter
# MotifSpace(length, rna)     length = value of m
m = 10
space = MotifSpace(m, rna=False)
allPossibleMers = space.motifsOfLength()
# list(allPossibleMers)

In [20]:
class ShiftScanner:
    def __init__(self, space: MotifSpace):
        assert(space.width <= 15)
        self.space = space
        
        global width
        width = space.width
#         print(width)
        mask = int('11'*width, 2)

        self.featuresByPriority = (Features(p, i, True) for p, i in enumerate(space.motifsOfLength()) )
    
    """ Find all matches in the string.
        Returns an array with the matches in order, or Motif.Empty for positions
        where no valid matches were found.
    """
    def allMatches(self, data, tolist=False):
        def getMatches(pos, motif):
            priority = encode(motif)
            return Motif(pos, Features(motif, priority, True))
        
        if tolist:
            return [(read[pos:pos+width], pos) for pos in range(len(read) - (width-1))]
        
        return (getMatches(pos, read[pos:pos+width]) for pos in range(len(read) - (width-1)))

In [9]:
read = "GACGACACCTATCTGCCCTCCGAAGAAACCGAGCATATGTCAGGCGGAGGGGTCGATGAGAGCGAGCTTGGAAGAAACGGCTTCCGCGAATCGGGGGGAGG"
scanner = ShiftScanner(space)
x = scanner.allMatches(read)
list(x)

10


[[0, (GACGACACCT, 549143)],
 [1, (ACGACACCTA, 99420)],
 [2, (CGACACCTAT, 397683)],
 [3, (GACACCTATC, 542157)],
 [4, (ACACCTATCT, 71479)],
 [5, (CACCTATCTG, 285918)],
 [6, (ACCTATCTGC, 95097)],
 [7, (CCTATCTGCC, 380389)],
 [8, (CTATCTGCCC, 472981)],
 [9, (TATCTGCCCT, 843351)],
 [10, (ATCTGCCCTC, 227677)],
 [11, (TCTGCCCTCC, 910709)],
 [12, (CTGCCCTCCG, 497110)],
 [13, (TGCCCTCCGA, 939864)],
 [14, (GCCCTCCGAA, 613728)],
 [15, (CCCTCCGAAG, 357762)],
 [16, (CCTCCGAAGA, 382472)],
 [17, (CTCCGAAGAA, 481312)],
 [18, (TCCGAAGAAA, 876672)],
 [19, (CCGAAGAAAC, 360961)],
 [20, (CGAAGAAACC, 395269)],
 [21, (GAAGAAACCG, 532502)],
 [22, (AAGAAACCGA, 32856)],
 [23, (AGAAACCGAG, 131426)],
 [24, (GAAACCGAGC, 525705)],
 [25, (AAACCGAGCA, 5668)],
 [26, (AACCGAGCAT, 22675)],
 [27, (ACCGAGCATA, 90700)],
 [28, (CCGAGCATAT, 362803)],
 [29, (CGAGCATATG, 402638)],
 [30, (GAGCATATGT, 561979)],
 [31, (AGCATATGTC, 150765)],
 [32, (GCATATGTCA, 603060)],
 [33, (CATATGTCAG, 315090)],
 [34, (ATATGTCAGG, 211786)],
 [3

In [23]:
class Node:
    def __init__(self, pos, motif: Motif):
        self.pos = pos
        self.rank = motif.feature.rank
        self.motif = motif
        self.next = None
        self.prev = None

    def __repr__(self):
        return '[%s, %s]' %(self.pos, self.motif)


class PosRankWindow:
    def __init__(self, size):
        self.head = None
        self.tail = None
        self.size = size
    
    def moveWindowAndInsert(self, pos: int, insertRight: Motif):
        new_node = Node(pos, insertRight)
        self.appendMonotonic(new_node, self.tail)
        self.dropUntilPosition(new_node)
        
        if pos - self.head.pos >= self.size:
            self.remove_head()
    
    def appendMonotonic(self, new_node, end):
        if end == None:
            self.insert_head(new_node)
        else:
            if new_node.rank < self.head.rank:
                self.insert_front(new_node)
            elif new_node.rank > end.rank:
                self.insert_after(new_node, end)
            else:
                self.appendMonotonic(new_node, end.prev)
    
    def insert_head(self, new_node):
        self.head = new_node
        self.head.next = None
        self.tail = new_node
        self.tail.prev = None

    def insert_front(self, new_node):
        new_node.prev = None
        new_node.next = self.head        
        self.head.prev = new_node
        self.head = new_node
    
    def insert_after(self, new_node, end):
        if end.next == None:
            end.next = new_node
            new_node.next = None
            new_node.prev = end
            self.tail = new_node
        else:
            new_node.next = end.next
            new_node.next.prev = new_node
            new_node.prev = end
            end.next = new_node

    def dropUntilPosition(self, insertedNode):
        if insertedNode.next != None:
            self.removeNext(insertedNode)
            self.dropUntilPosition(insertedNode)
    
    def removeNext(self, insertedNode):
        temp = insertedNode.next
        if temp.next != None:
            insertedNode.next = temp.next
            temp.next.prev = insertedNode
        else:
            insertedNode.next = None
            self.tail = insertedNode
            temp.prev = None
        
        del temp
        
    def remove_head(self):
        if self.head == None:
            print("List is empty")
        else:
            temp = self.head
            temp.next.prev = None                                       
            self.head = temp.next
            temp.next = None
            del temp
    
    def top(self): 
        if self.head == None:
            print("Window is empty")
        else:
            return self.head

    def show_window(self):
        temp = self.head
        print(temp.pos, temp.motif)
        while temp != None:
            print([temp.pos, temp.motif])
            temp = temp.next

In [12]:
class MotifExtractor:
    def __init__(self, space: MotifSpace, k: int):
        self.scanner = space.scanner()
        self.K = k
        
        global width
        width = space.width

    def slidingTopMotifs(self, read):
        matches = self.scanner.allMatches(read)
        windowMotifs = PosRankWindow(self.K - (width - 1))

        if (len(read) < self.K):
            return iter()
        else:
            pos = width - self.K
            for m in matches:
                windowMotifs.moveWindowAndInsert(pos, m)
                pos += 1
                if pos > 0: yield windowMotifs.top()
    
    def regionsInRead(self, read):
        topMotifs = self.slidingTopMotifs(read)
        lastMotif = next(topMotifs)

        consumed = 1
        startReg = 1
        for motif in topMotifs:
            if lastMotif == motif:
                consumed += 1
            else:
                yield lastMotif, startReg-consumed
                lastMotif = motif
                consumed = 1

            startReg += 1
        
        # for the last motif
        yield lastMotif, startReg - consumed

    def splitRead(self, read):
        readByReg = self.regionsInRead(read)
        prev = next(readByReg)
        while readByReg:
            b1 = prev
            b2 = next(readByReg, None)
#             print(b1, b2)
            if b2:
                yield b1[0], read[ b1[1] : b2[1] + (self.K - 1) ]
                prev = b2
            else:
                yield b1[0], read[b1[1] : ]
                break
            

In [26]:
def main(read):
    m = 10
    k = 28
    space = MotifSpace(m, False)
    space.motifsOfLength()
    
    ex = MotifExtractor(space, k)
    return ex.splitRead(read)
    

In [30]:
file = open('data/testData.fasta')
for data in file.readlines():
    read = data.strip('\n')
    if read[0] != '>':
        print('Read :',read)
        for x in main(read):
            print(x)
        print('\n')
file.close()

Read : TGGCGTCATTTTCCCCAATCATAGACTGTTGAAAGTGAACAAAACTGATGACTATCGTGCCACATTGATTGTTGTCGGTGCCTCGCTTCTGAACGACCCCT
([-2, [16, (AATCATAGAC, 54049)]], 'TGGCGTCATTTTCCCCAATCATAGACTGTTGAAAGTGAAC')
([13, [31, (AAAGTGAACA, 11780)]], 'CCCAATCATAGACTGTTGAAAGTGAACAAAACTGAT')
([22, [40, (AAAACTGATG, 1934)]], 'AGACTGTTGAAAGTGAACAAAACTGATGACTATCGTGCCACATTGA')
([23, [41, (AAACTGATGA, 7736)]], 'AAACTGATGACTATCGTGCCACATTGAT')
([24, [42, (AACTGATGAC, 30945)]], 'AACTGATGACTATCGTGCCACATTGATT')
([43, [61, (ACATTGATTG, 81470)]], 'ACTGATGACTATCGTGCCACATTGATTGTTGTCGGTGCCTCGCTTC')
([45, [63, (ATTGATTGTT, 254959)]], 'CATTGATTGTTGTCGGTGCCTCGCTTCTG')
([49, [67, (ATTGTTGTCG, 257974)]], 'TTGATTGTTGTCGGTGCCTCGCTTCTGAACG')
([62, [80, (CCTCGCTTCT, 383479)]], 'TTGTTGTCGGTGCCTCGCTTCTGAACGACCCC')
([73, [91, (AACGACCCCT, 24919)]], 'GTCGGTGCCTCGCTTCTGAACGACCCCT')


Read : CCCGAATACAAAATTGGTCTTTGGCGAAACGATTGGAAACCCGGCGCTTGGCGTTCTGGATTTCGAAAAGTTCTCGAAGCTTGCCAAGGAATTCGATGTGC
([-9, [9, (AAAATTGGTC, 4013)]], 'CCCGAATACAAAATTGGTCTTT